# Analysis of scATAC-seq data in PBMC using Signac


In this note, the data is from 10x. Cellranger report can be found in the link below: 

https://cf.10xgenomics.com/samples/cell-atac/2.1.0/10k_pbmc_ATACv2_nextgem_Chromium_Controller/10k_pbmc_ATACv2_nextgem_Chromium_Controller_web_summary.html


## Data downloading 

To download all the required files, you can run the following lines in a shell:

In [3]:
getwd()

[1] "/mnt/ccrsf-static/Analysis/xies4/github_repos/ngs_software_testing_notes/docs"

In [27]:
cat(system("cat ./scatac_signac_pbmc/s1_dl.sh", intern = T), sep = "\n")

#!/bin/bash
wget https://cf.10xgenomics.com/samples/cell-atac/2.1.0/10k_pbmc_ATACv2_nextgem_Chromium_Controller/10k_pbmc_ATACv2_nextgem_Chromium_Controller_filtered_peak_bc_matrix.h5
wget https://cf.10xgenomics.com/samples/cell-atac/2.1.0/10k_pbmc_ATACv2_nextgem_Chromium_Controller/10k_pbmc_ATACv2_nextgem_Chromium_Controller_singlecell.csv
wget https://cf.10xgenomics.com/samples/cell-atac/2.1.0/10k_pbmc_ATACv2_nextgem_Chromium_Controller/10k_pbmc_ATACv2_nextgem_Chromium_Controller_fragments.tsv.gz
wget https://cf.10xgenomics.com/samples/cell-atac/2.1.0/10k_pbmc_ATACv2_nextgem_Chromium_Controller/10k_pbmc_ATACv2_nextgem_Chromium_Controller_fragments.tsv.gz.tbi


In [6]:
cat(system("head 10k_pbmc_ATACv2_nextgem_Chromium_Controller_singlecell.csv", intern = T), sep="\n")

barcode,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,nonprimary,passed_filters,is__cell_barcode,excluded_reason,TSS_fragments,DNase_sensitive_region_fragments,enhancer_region_fragments,promoter_region_fragments,on_target_fragments,blacklist_region_fragments,peak_region_fragments,peak_region_cutsites
NO_BARCODE,20081988,3972351,3423,2118408,1159348,23059,4131,12801268,0,0,0,0,0,0,0,0,0,0
AAACGAAAGAAACGCC-1,4,0,0,0,0,0,0,4,0,0,1,0,0,0,1,0,2,4
AAACGAAAGAAAGCAG-1,3,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,1,2
AAACGAAAGAAAGGGT-1,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,2
AAACGAAAGAAATACC-1,17,0,0,0,0,0,0,17,0,0,11,0,0,0,11,0,14,27
AAACGAAAGAAATCTG-1,1,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0
AAACGAAAGAAATGGG-1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
AAACGAAAGAAATTCG-1,4,0,0,0,1,0,0,3,0,0,1,0,0,0,1,0,2,4
AAACGAAAGAACAGGA-1,2,0,0,0,0,0,0,2,0,0,2,0,0,0,2,0,2,4


## Load R packages

In [2]:
library(Signac)
library(Seurat)
library(ggplot2)
library(patchwork)
library(GenomicRanges)

## Data preprocessing 

Signac uses information from two related input files, both of which can be created using CellRanger:

1. Peak/Cell matrix. This is analogous to the gene expression count matrix used to analyze single-cell RNA-seq. However, instead of genes, each row of the matrix represents a region of the genome (a peak), that is predicted to represent a region of open chromatin. Each value in the matrix represents the number of Tn5 integration sites for each single barcode (i.e. a cell) that map within each peak. You can find more detail on the 10X Website.

2. Fragment file. This represents a full list of all unique fragments across all single cells. It is a substantially larger file, is slower to work with, and is stored on-disk (instead of in memory). However, the advantage of retaining this file is that it contains all fragments associated with each single cell, as opposed to only fragments that map to peaks. More information about the fragment file can be found on the 10x Genomics website or on the sinto website.



In [3]:
counts <- Read10X_h5(filename = "10k_pbmc_ATACv2_nextgem_Chromium_Controller_filtered_peak_bc_matrix.h5")

In [4]:
metadata <- read.csv(
  file = "10k_pbmc_ATACv2_nextgem_Chromium_Controller_singlecell.csv",
  header = TRUE,
  row.names = 1
)

In [7]:
chrom_assay <- CreateChromatinAssay(
  counts = counts,
  sep = c(":", "-"),
  fragments = "10k_pbmc_ATACv2_nextgem_Chromium_Controller_fragments.tsv.gz",
  min.cells = 10,
  min.features = 200
)


Computing hash



In [8]:
pbmc <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks",
  meta.data = metadata
)

In [9]:
pbmc

An object of class Seurat 
165434 features across 10246 samples within 1 assay 
Active assay: peaks (165434 features, 0 variable features)
 2 layers present: counts, data

Similar to scRNA-seq, you can also read the count matrix stored in three files under the folder: `filtered_peak_bc_matrix`: 

```
counts <- Matrix::readMM("filtered_peak_bc_matrix/matrix.mtx")
barcodes <- readLines("filtered_peak_bc_matrix/barcodes.tsv")
peaks <- read.table("filtered_peak_bc_matrix/peaks.bed", sep="\t")
peaknames <- paste(peaks$V1, peaks$V2, peaks$V3, sep="-")
colnames(counts) <- barcodes
rownames(counts) <- peaknames
```


In [10]:
pbmc[['peaks']]

ChromatinAssay data with 165434 features for 10246 cells
Variable features: 0 
Genome: 
Annotation present: FALSE 
Motifs present: FALSE 
Fragment files: 1 

In [11]:
granges(pbmc)


GRanges object with 165434 ranges and 0 metadata columns:
             seqnames        ranges strand
                <Rle>     <IRanges>  <Rle>
       [1]       chr1    9772-10660      *
       [2]       chr1 180712-181178      *
       [3]       chr1 181200-181607      *
       [4]       chr1 191183-192084      *
       [5]       chr1 267576-268461      *
       ...        ...           ...    ...
  [165430] KI270713.1   13054-13909      *
  [165431] KI270713.1   15212-15933      *
  [165432] KI270713.1   21459-22358      *
  [165433] KI270713.1   29676-30535      *
  [165434] KI270713.1   36913-37813      *
  -------
  seqinfo: 35 sequences from an unspecified genome; no seqlengths

We then remove the features that correspond to chromosome scaffolds e.g. (KI270713.1) or other sequences instead of the (22+2) standard chromosomes.

The function `standardChromosomes` in R package `GenomeInfoDb` lists the 'standard' chromosomes defined as sequences in the assembly that are not scaffolds; also referred to as an 'assembly molecule' in NCBI.

In [14]:
standardChromosomes(granges(pbmc))

[1] "chr1"  "chr2"  "chr3"  "chr4"  "chr5"  "chr6"  "chr7"  "chr8"  "chr9" 
[10] "chr10" "chr11" "chr12" "chr13" "chr14" "chr15" "chr16" "chr17" "chr18"
[19] "chr19" "chr20" "chr21" "chr22" "chrX"  "chrY"

In [12]:
peaks.keep <- seqnames(granges(pbmc)) %in% standardChromosomes(granges(pbmc))
pbmc <- pbmc[as.vector(peaks.keep), ]

In [17]:
library(AnnotationHub)
ah <- AnnotationHub()

# Search for the Ensembl 98 EnsDb for Homo sapiens on AnnotationHub
query(ah, "EnsDb.Hsapiens.v98")

Loading required package: BiocFileCache

Loading required package: dbplyr



AnnotationHub with 1 record
# snapshotDate(): 2023-10-23
# names(): AH75011
# $dataprovider: Ensembl
# $species: Homo sapiens
# $rdataclass: EnsDb
# $rdatadateadded: 2019-05-02
# $title: Ensembl 98 EnsDb for Homo sapiens
# $description: Gene and protein annotations for Homo sapiens based on Ensem...
# $taxonomyid: 9606
# $genome: GRCh38
# $sourcetype: ensembl
# $sourceurl: http://www.ensembl.org
# $sourcesize: NA
# $tags: c("98", "AHEnsDbs", "Annotation", "EnsDb", "Ensembl", "Gene",
#   "Protein", "Transcript") 
# retrieve record with 'object[["AH75011"]]' 

## Session Infomation

In [2]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Oracle Linux Server 8.10

Matrix products: default
BLAS/LAPACK: /home/xies4/micromamba/envs/condaenv4bix_tool_notes_scatac/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: America/New_York
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] digest_0.6.36   IRdisplay_1.1   utf8_1.2.4      base64enc_0.1-3
 [5] fastmap_1.2.0   glue_1.7.0      htmltools_0.5.7 repr_1.1.6     
 [9] lifecycle_1.0.4 cli_3.6.3      

## Reference 




In [ ]:
https://stuartlab.org/signac/articles/pbmc_vignette